In [7]:
from nilearn import plotting
from nilearn import image
import matplotlib

path = "c:/Users/haris/Desktop/fMRT Datensatz/T1w/Results/rfMRI_REST1_LR/PhaseOne_gdc_dc.nii.gz"
img = image.load_img(path) 
Dataset = img.get_fdata()
print("Data shape:", Dataset.shape)


Data shape: (260, 311, 260, 3)


In [4]:
time_point = 2
Data_slice = Dataset[:, :, :, time_point]
#plotting.plot_img(Data_slice)
print(Data_slice.shape)


(260, 311, 260)


In [ ]:
plotting.plot_glass_brain(Data_slice, threshold=3)

In [9]:
first_Dataset = image.index_img(Dataset, 0)
print(first_Dataset.shape)

TypeError: Data given cannot be loaded because it is not compatible with nibabel format:
-0.0